In [ ]:
!pip install datasets torchaudio jiwer

In [14]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

device = 'cuda:0'

processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small").to(device)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [2]:
import torch
torch.backends.quantized.engine = 'fbgemm'

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

torch.save(quantized_model, 'qwisper.pth')

In [17]:
import os

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [18]:
print_size_of_model(model)

Size (MB): 967.089099


In [19]:
print_size_of_model(quantized_model)

Size (MB): 412.528252


In [12]:
from datasets import load_dataset

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [20]:
ds = load_dataset("mozilla-foundation/common_voice_13_0", "ru", split="train")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6752it [00:00, 67511.86it/s]
Reading metadata...: 26269it [00:00, 100686.72it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...


Reading metadata...: 10186it [00:00, 113980.76it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13491it [00:00, 105451.26it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9523it [00:00, 87112.05it/s]


In [22]:
import torchaudio

In [23]:
def get_array(sample):
    sampling_rate = sample['sampling_rate']
    array = torch.tensor(sample['array'])
    if sampling_rate != 16000:
        array = torchaudio.functional.resample(array, sampling_rate, 16000)
    return array


In [25]:
from jiwer import wer
from tqdm import tqdm
import time

In [52]:
q_time, n_time = 0, 0
q_wer, n_wer = 0, 0
k = 30
audio_duration = 0
for n in tqdm(range(k)):
    sample = ds[n]["audio"]
    text = ds[n]['sentence']
    arr = get_array(sample)
    input_features = processor(arr, sampling_rate=16000, return_tensors="pt").input_features.to(device)
    audio_duration += len(arr)/16000


    start = time.time()
    predicted_ids = model.generate(input_features, language='ru')
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True, language='ru')[0].strip()
    end = time.time()

    n_time += end-start
    n_wer += wer(text, transcription)

100%|██████████| 30/30 [00:19<00:00,  1.54it/s]


In [53]:
print('Whisper small on russian corpus of common_voice_13_0 dataset')
print(f'Device: {torch.cuda.get_device_name()}')
print(f'Mean WER on {k} samples: {(n_wer / k):.4}')
print(f'Mean RTF on {k} samples: {(n_time / audio_duration):.4}')

Whisper small on russian corpus of common_voice_13_0 dataset
Device: Tesla T4
Mean WER on 30 samples: 0.3401
Mean RTF on 30 samples: 0.1012


In [58]:
i = 20
sample = ds[i]["audio"]
text = ds[i]['sentence']
arr = get_array(sample)
input_features = processor(arr, sampling_rate=16000, return_tensors="pt").input_features.to(device)

predicted_ids = model.generate(input_features, language='ru')
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True, language='ru')[0].strip()

print(f'Predicted text: {transcription}')
print(f'     True text: {text}')

Predicted text: Финансирование миссии организации безненахнации поделаем время на администрацию в Косово.
     True text: Финансирование Миссии Организации Объединенных Наций по делам временной администрации в Косово.
